In [1]:
import openrouteservice
import pandas as pd
import os
import time
import numpy as np

# Chave da API
ORS_API_KEY = "5b3ce3597851110001cf624882d5a1b8d3444c489b1b384e680e6093"
client = openrouteservice.Client(key=ORS_API_KEY)


input_dir = "circuitos_individuais"
output_dir = "MATRIZES"
os.makedirs(output_dir, exist_ok=True)

In [2]:
BLOCK_SIZE = 50

def get_block_distance(orig_coords, dest_coords):
    """Faz uma chamada à API para obter a submatriz entre origens e destinos"""
    try:
        result = client.distance_matrix(
            locations=orig_coords + dest_coords,
            sources=list(range(len(orig_coords))),
            destinations=list(range(len(orig_coords), len(orig_coords) + len(dest_coords))),
            profile='driving-car',
            metrics=['distance'],
            units='m'
        )
        return np.array(result['distances'])
    except Exception as e:
        print("Erro na chamada à API:", e)
        return np.full((len(orig_coords), len(dest_coords)), np.nan)

# Obter e ordenar os ficheiros CSV
csv_files = sorted([
    f for f in os.listdir(input_dir)
    if f.endswith(".csv")
])

# Contador para os Circuitos normais
contador_circuito = 1

# Processar todos os ficheiros
for file_name in csv_files:
    df = pd.read_csv(os.path.join(input_dir, file_name))
    coords = [(row['lon'], row['lat']) for _, row in df.iterrows()]
    N = len(coords)

    print(f"A criar matriz para {file_name} com {N} pontos...")

    # Inicializar matriz vazia
    full_matrix = np.zeros((N, N))

    # Dividir em blocos e fazer chamadas
    for i in range(0, N, BLOCK_SIZE):
        for j in range(0, N, BLOCK_SIZE):
            block_orig = coords[i:i+BLOCK_SIZE]
            block_dest = coords[j:j+BLOCK_SIZE]
            block_dist = get_block_distance(block_orig, block_dest)
            full_matrix[i:i+len(block_orig), j:j+len(block_dest)] = block_dist
            time.sleep(1)

    # Verificar nome especial
    if file_name.lower() == "circ_ilhas_pontos.csv":
        out_name = "matriz_dist_Circ_Ilhas.csv"
    else:
        out_name = f"matriz_dist_Circuito_{contador_circuito:02d}.csv"
        contador_circuito += 1

    out_path = os.path.join(output_dir, out_name)

    # Guardar matriz sem cabeçalhos nem índices
    pd.DataFrame(full_matrix).to_csv(out_path, index=False, header=False)

    print(f"Matriz guardada: {out_path}")

A criar matriz para circ_01_pontos.csv com 66 pontos...
Matriz guardada: MATRIZES\matriz_dist_Circuito_01.csv
A criar matriz para circ_02_pontos.csv com 72 pontos...
Matriz guardada: MATRIZES\matriz_dist_Circuito_02.csv
A criar matriz para circ_03_pontos.csv com 67 pontos...
Matriz guardada: MATRIZES\matriz_dist_Circuito_03.csv
A criar matriz para circ_04_pontos.csv com 72 pontos...
Matriz guardada: MATRIZES\matriz_dist_Circuito_04.csv
A criar matriz para circ_05_pontos.csv com 75 pontos...
Matriz guardada: MATRIZES\matriz_dist_Circuito_05.csv
A criar matriz para circ_06_pontos.csv com 75 pontos...
Matriz guardada: MATRIZES\matriz_dist_Circuito_06.csv
A criar matriz para circ_07_pontos.csv com 69 pontos...
Matriz guardada: MATRIZES\matriz_dist_Circuito_07.csv
A criar matriz para circ_08_pontos.csv com 75 pontos...
Matriz guardada: MATRIZES\matriz_dist_Circuito_08.csv
A criar matriz para circ_09_pontos.csv com 50 pontos...
Matriz guardada: MATRIZES\matriz_dist_Circuito_09.csv
A criar ma

C:\Users\simao\anaconda5\envs\maenvs\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\Users\simao\anaconda5\envs\maenvs\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\Users\simao\anaconda5\envs\maenvs\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\Users\simao\anaconda5\envs\maenvs\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\Users\simao\anaconda5\env

Matriz guardada: MATRIZES\matriz_dist_Circuito_11.csv
A criar matriz para circ_12_pontos.csv com 144 pontos...
Matriz guardada: MATRIZES\matriz_dist_Circuito_12.csv
A criar matriz para circ_ilhas_pontos.csv com 44 pontos...
Matriz guardada: MATRIZES\matriz_dist_Circ_Ilhas.csv


In [4]:

BLOCK_SIZE = 50  # Máximo seguro para evitar limite de 3500 rotas

def get_block_duration(orig_coords, dest_coords):
    """Chamada à API para obter submatriz de tempo entre origens e destinos"""
    try:
        result = client.distance_matrix(
            locations=orig_coords + dest_coords,
            sources=list(range(len(orig_coords))),
            destinations=list(range(len(orig_coords), len(orig_coords) + len(dest_coords))),
            profile='driving-car',
            metrics=['duration'],  # Tempo em segundos
        )
        return np.array(result['durations'])
    except Exception as e:
        print("Erro na chamada à API:", e)
        return np.full((len(orig_coords), len(dest_coords)), np.nan)

# Obter e ordenar os ficheiros CSV
csv_files = sorted([
    f for f in os.listdir(input_dir)
    if f.endswith(".csv")
])

# Contador para os Circuitos normais
contador_circuito = 1

# Processar todos os ficheiros
for file_name in csv_files:
    df = pd.read_csv(os.path.join(input_dir, file_name))
    coords = [(row['lon'], row['lat']) for _, row in df.iterrows()]
    N = len(coords)

    print(f"A criar matriz de tempo para {file_name} com {N} pontos...")

    # Inicializar matriz vazia
    full_matrix = np.zeros((N, N))

    # Dividir em blocos e fazer chamadas
    for i in range(0, N, BLOCK_SIZE):
        for j in range(0, N, BLOCK_SIZE):
            block_orig = coords[i:i+BLOCK_SIZE]
            block_dest = coords[j:j+BLOCK_SIZE]
            block_dur = get_block_duration(block_orig, block_dest)
            full_matrix[i:i+len(block_orig), j:j+len(block_dest)] = block_dur
            time.sleep(1)

    # Converter de segundos para minutos
    full_matrix_minutes = full_matrix / 60

    # Verificar nome especial
    if file_name.lower() == "circ_ilhas_pontos.csv":
        out_name = "matriz_tempo_Circ_Ilhas.csv"
    else:
        out_name = f"matriz_tempo_Circuito_{contador_circuito:02d}.csv"
        contador_circuito += 1

    out_path = os.path.join(output_dir, out_name)

    # Guardar matriz em minutos sem cabeçalhos nem índices
    pd.DataFrame(full_matrix_minutes).to_csv(out_path, index=False, header=False)

    print(f"Matriz guardada (em minutos): {out_path}")

A criar matriz de tempo para circ_01_pontos.csv com 66 pontos...
Matriz guardada (em minutos): MATRIZES\matriz_tempo_Circuito_01.csv
A criar matriz de tempo para circ_02_pontos.csv com 72 pontos...
Matriz guardada (em minutos): MATRIZES\matriz_tempo_Circuito_02.csv
A criar matriz de tempo para circ_03_pontos.csv com 67 pontos...
Matriz guardada (em minutos): MATRIZES\matriz_tempo_Circuito_03.csv
A criar matriz de tempo para circ_04_pontos.csv com 72 pontos...
Matriz guardada (em minutos): MATRIZES\matriz_tempo_Circuito_04.csv
A criar matriz de tempo para circ_05_pontos.csv com 75 pontos...
Matriz guardada (em minutos): MATRIZES\matriz_tempo_Circuito_05.csv
A criar matriz de tempo para circ_06_pontos.csv com 75 pontos...
Matriz guardada (em minutos): MATRIZES\matriz_tempo_Circuito_06.csv
A criar matriz de tempo para circ_07_pontos.csv com 69 pontos...
Matriz guardada (em minutos): MATRIZES\matriz_tempo_Circuito_07.csv
A criar matriz de tempo para circ_08_pontos.csv com 75 pontos...
Matr

C:\Users\simao\anaconda5\envs\maenvs\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\Users\simao\anaconda5\envs\maenvs\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\Users\simao\anaconda5\envs\maenvs\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\Users\simao\anaconda5\envs\maenvs\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\Users\simao\anaconda5\env

Matriz guardada (em minutos): MATRIZES\matriz_tempo_Circuito_11.csv
A criar matriz de tempo para circ_12_pontos.csv com 144 pontos...
Matriz guardada (em minutos): MATRIZES\matriz_tempo_Circuito_12.csv
A criar matriz de tempo para circ_ilhas_pontos.csv com 44 pontos...
Matriz guardada (em minutos): MATRIZES\matriz_tempo_Circ_Ilhas.csv
